In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [17]:
import importlib

In [2]:
from src import elastic_search_engine, rag

In [18]:
importlib.reload(elastic_search_engine)
importlib.reload(rag)

<module 'src.rag' from '/home/taras/my_code_for_courses/llm-zoomcamp/Project/src/rag.py'>

In [12]:
import pandas as pd
import json

In [14]:
with open('../data/initial_data_w_id.json', 'r') as f_in:
    documents = json.load(f_in)

In [16]:
doc_idx = {d['id']: d for d in documents}

# Example of a possible RAG

In [20]:
elastic_semantic_searcher = elastic_search_engine.ElasticSemanticSearcher(index_name='vague-actual-mpnet')
prompt_template = """
    Your are a translator from vague boss language into an everyday language. Translate
    the VAGUE statement or question based on the CONTEXT. Provide a clear and concise answer.
    VAGUE: {vague}
    
    CONTEXT: {context}
    """.strip()
    
chatgpt_4o_mini = rag.ChatGPTRAG(elastic_searcher=elastic_semantic_searcher,
                 prompt_template=prompt_template, 
                 llm_model='gpt-4o-mini',
                 sentence_transformer_name='all-mpnet-base-v2')

/home/taras/.cache/pypoetry/virtualenvs/src-SOtueW5z-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
chatgpt_4o_mini.rag_results('We need to become Speedy Gonzales and kick it into hyperdrive or we are going to be stuck in traffic.')

'We need to accelerate our progress significantly to avoid missing important deadlines.'

## Evaluation for different RAGs

In [7]:
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor


In [11]:
ground_truth = pd.read_csv('../data/ground_truth_data.csv')

In [22]:
ground_truth = ground_truth.to_dict(orient='records')

In [8]:
pool = ThreadPoolExecutor(max_workers=6)

In [9]:
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [28]:
def process_record(rec):
    model = chatgpt_4o_mini
    answer_llm = model.rag_results(rec['vague'])
    
    doc_id = rec['doc_id']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['actual']

    return {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['vague'],
    }

In [30]:
process_record(ground_truth[10])

{'answer_llm': 'We need to motivate our team to be more creative and come up with new ideas.',
 'answer_orig': 'We need to encourage creativity to drive innovation within our team.',
 'document': '19b2160f076bcc9c5a36cd7f39291c1b',
 'question': 'Creativity needs a spark, so let’s stoke the fire and unleash some innovative thinking.'}

In [29]:
result_prompt_standard = map_progress(pool, ground_truth, process_record)

  0%|          | 0/4075 [00:00<?, ?it/s]

KeyboardInterrupt: 